In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras.callbacks import TensorBoard
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [3]:
df = pd.read_csv('../train.csv')
print(df.shape)
df.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
df_train = df.copy()
# df_train['month'] = pd.DatetimeIndex(df_train['datetime']).month
# df_train = df[df['datetime'] < '2011-02'].copy()
# df_train = df[(df['datetime'] > month_ini) & (df['datetime'] < month_end)].copy()
df_train['day'] = pd.DatetimeIndex(df_train['datetime']).day
df_train['dayofweek'] = pd.DatetimeIndex(df_train['datetime']).dayofweek
df_train['hour'] = pd.DatetimeIndex(df_train['datetime']).hour
df_train = df_train.drop('datetime', axis=1)
y = df_train['count'].copy()
df_train = df_train.drop(['casual','registered','count'], axis=1)
# df = df.drop(['casual','registered'], axis=1)
print(df_train.shape)
df_train.head()

(10886, 12)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,month,day,dayofweek,hour
0,1,0,0,1,9.84,14.395,81,0.0,1,1,5,0
1,1,0,0,1,9.02,13.635,80,0.0,1,1,5,1
2,1,0,0,1,9.02,13.635,80,0.0,1,1,5,2
3,1,0,0,1,9.84,14.395,75,0.0,1,1,5,3
4,1,0,0,1,9.84,14.395,75,0.0,1,1,5,4


In [15]:
X = df_train.values
Y = y.values
maxlen = 20
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape(len(X), 1, 12)
def new_model():
    n_features = 12
    # create model
    model = Sequential()
    model.add(LSTM(50, input_shape=(1, n_features), return_sequences=True, activation='relu'))
    model.add(LSTM(50, input_shape=(1, n_features), return_sequences=True, activation='relu'))
    model.add(LSTM(50, input_shape=(1, n_features), activation='relu'))
    model.add(Dense(1))
    model.add(Activation('linear'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
kr = KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=2)
%time kr.fit(X, Y, callbacks=[TensorBoard(log_dir='logs', histogram_freq=10)])

Epoch 1/100
34s - loss: 24776.4331
Epoch 2/100
32s - loss: 18386.3862
Epoch 3/100
32s - loss: 13897.9259
Epoch 4/100
33s - loss: 11858.3711
Epoch 5/100
31s - loss: 10139.5931
Epoch 6/100
31s - loss: 8438.7599
Epoch 7/100
31s - loss: 7097.3856
Epoch 8/100
34s - loss: 6219.9943
Epoch 9/100
32s - loss: 5623.3778
Epoch 10/100
33s - loss: 5361.9884
Epoch 11/100
33s - loss: 5080.8399
Epoch 12/100
32s - loss: 4915.0932
Epoch 13/100
31s - loss: 4807.3321
Epoch 14/100
31s - loss: 4694.1351
Epoch 15/100
32s - loss: 4649.9606
Epoch 16/100
34s - loss: 4444.0220
Epoch 17/100
35s - loss: 4374.0845
Epoch 18/100
34s - loss: 4249.7759
Epoch 19/100
35s - loss: 4203.6557
Epoch 20/100
33s - loss: 4125.2781
Epoch 21/100
35s - loss: 3956.0768
Epoch 22/100
34s - loss: 3900.1377
Epoch 23/100
32s - loss: 3905.0368
Epoch 24/100
33s - loss: 3824.8357
Epoch 25/100
33s - loss: 3744.3529
Epoch 26/100
31s - loss: 3687.6328
Epoch 27/100
32s - loss: 3676.3786
Epoch 28/100
32s - loss: 3579.1980
Epoch 29/100
31s - loss:

## Prediction

In [8]:
df_test = pd.read_csv('../test.csv')
df_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [1]:
dft = df_test.copy()
dft['month'] = pd.DatetimeIndex(dft['datetime']).month
# dft = df_test[df_test['datetime'] < '2011-02'].copy()
# dft = df_test[(df_test['datetime'] > month_ini) & (df_test['datetime'] < month_end)].copy()
dft['day'] = pd.DatetimeIndex(dft['datetime']).day
dft['dayofweek'] = pd.DatetimeIndex(dft['datetime']).dayofweek
dft['hour'] = pd.DatetimeIndex(dft['datetime']).hour
dft = dft.drop('datetime', axis=1)
print(dft.shape)
dft.head()

NameError: name 'df_test' is not defined

In [ ]:
dft = scaler.transform(dft)
dft = dft.reshape(len(dft), 1, 12)
test_pred = kr.predict(dft)
test_pred[test_pred < 0] = 0

str_prediction = "datetime,count\n"
for i in range(0, len(test_pred)):
    datetime = df_test['datetime'][i]
    pred = int(round(test_pred[i]))
    str_prediction += "{},{}\n".format(datetime, pred)

# print(str_prediction)
f = open('pred_keras_lstm.csv','w')
f.write(str_prediction)
f.close()